In [7]:
import os
os.environ["MXNET_CUDNN_LIB_CHECKING"] = "0"
os.environ["MXNET_CUDNN_AUTOTUNE_DEFAULT"] = "0"
import time
import random
import numpy as np
import mxnet as mx
from mxnet import  autograd, context
from mxnet.base import MXNetError
from sklearn.model_selection import train_test_split

import myModel
from image_Dictionary import ImageDict
import const
from my_Save import saveAsCSV, SaveModels
from mx_Train import myTrain
from decode.postprocessing.instance_segmentation import InstSegm
from myPlots import lossPlot, visualize_all, plot_images_with_masks, plotPredictedImape, plotIOUS
from iou import  get_iou

ctx=context.gpu()
mx.nd.waitall()
numberOfimages = 648

isVnir = False
imageType = "NDV"
if isVnir:
    imageType = "VNIR"

input_directory = const.images_2022
output_directory = os.path.join(const.result_2022, imageType, str(numberOfimages))
output_models= os.path.join(output_directory,"models")
result_path = os.path.join(output_directory, "result")
lossFile =   os.path.join(output_directory,"loss.csv") 
#print('imageType:', imageType)

def makedir(path):
    os.makedirs(path, exist_ok=True)
    #print(path)
    
for i in [output_directory, output_models, result_path]:
    makedir(i)

# Function to load images and masks and Preprocessing

In [ ]:
images_2022= ImageDict(const.images_2022,  False)
image_dict_2022  = images_2022.load_tif_files(imageType, numberOfimages = numberOfimages)
masks_2022 = ImageDict(const.masks_2022, True)
mask_dict_2022  = masks_2022.load_tif_files(imageType, image_dict_2022, numberOfimages = numberOfimages)
train_ids, val_ids = train_test_split(list(mask_dict_2022.keys()), test_size=0.2, random_state=42)


In [9]:
images_2010 = ImageDict(const.images_2010, False)
testimages = 648
image_dict_2010 = images_2010.load_tif_files(imageType, numberOfimages=testimages)
output_directory_2010 = os.path.join(const.result_2010, imageType)
makedir(output_directory_2010)

In [4]:
def get_images(ids, image_dict):
    data=  np.array([image_dict[id].image for id in ids])
    return mx.nd.array(data)

train_images =get_images(train_ids, image_dict_2022)
train_masks = get_images(train_ids, mask_dict_2022)
val_images = get_images(val_ids, image_dict_2022)
val_masks = get_images(val_ids, mask_dict_2022)

batch_size=4
try:
    train_dataset = mx.gluon.data.ArrayDataset(train_images, train_masks)
    train_loader = mx.gluon.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0,shuffle=True)
    val_loader = mx.gluon.data.DataLoader(mx.gluon.data.ArrayDataset(val_images, val_masks), batch_size=batch_size, num_workers=0, shuffle=False)

except Exception as e:
    print(f"Error creating data loaders: {e}")

In [ ]:
# Maximum number of retries
max_retries = 5
retry_count = 0

# Retry logic
while retry_count < max_retries:
    try:
        mxTn = myTrain(train_loader, val_loader)
        loss_each_epoch, model_list, epoch = mxTn.train(ctx, epochs = 50)
        saveAsCSV(["Current Epoch", "Training Loss", "Validation Loss"], lossFile, loss_each_epoch)
        SaveModels(output_models, model_list)
        lossPlot(loss_each_epoch, output_directory)
        break
    except MXNetError  as e:
        if 'CUDNN_STATUS_EXECUTION_FAILED' in str(e):
            print(f"cuDNN execution failed. Retrying... ({retry_count + 1}/{max_retries})")
            mx.nd.waitall()  # Clear GPU memory
            time.sleep(5) # Wait for a few seconds before retrying
            retry_count += 1 # Increment the retry counter
        else:
            raise  # If it's another error, raise it

# Check if maximum retries were reached
if retry_count == max_retries:
    print("Maximum retries reached. Training failed due to cuDNN error.")

In [10]:
def get_model_file_name():
    files = os.listdir(output_models)  # Get all files in the folder
    if files:
        last_file = os.path.join(output_models, f'model_VNIR_{len(files)-1}.params')
        print(f"This model is using: {last_file}")
        return last_file
    else:
        print("The folder is empty.")

def get_img_metadata(id, is2022):
    if is2022:
        img = images_2022.getImage(id, image_dict_2022, ctx)
        mask = masks_2022.getImage(id, mask_dict_2022, ctx)
        currentMetadata = image_dict_2022[id]
    else:
        img = images_2010.getImage(id, image_dict_2010, ctx)
        currentMetadata = image_dict_2010[id]
        mask = None
    return img, mask, currentMetadata

def get_ref_path(is2022):
    ref_path = const.output_ref_2022    
    if not is2022:
        ref_path = const.output_ref_2010
    return ref_path

def visualize_predictions(result_path, val_ids, t_ext , t_bound , is2022):    
    print(f"Starting visualization with t_ext = {t_ext}, t_bound = {t_bound}")
    modelPath = rf"{get_model_file_name()}"
    ref_path = get_ref_path(is2022)   

    netPredict = myModel.MyFractalResUNetcmtsk(True, modelPath, ctx)
    ious=[]
    ious.append({"ID": f't_ext: {t_ext}',"IOU": f't_bound: {t_bound}'}) 
    plotColl = []

    for id in val_ids:  # Limit to 'num_images' for visualization
        print(f"Processing image ID: {id}")
        try:
            img, mask, currentMetadata = get_img_metadata(id, is2022)
           
            with autograd.predict_mode():  
                outputs = netPredict.net(img) 
                pred_segm  = np.array(outputs[0][0,1,:,:].asnumpy())
                pred_bound =  np.array(outputs[1][0,1,:,:].asnumpy())
                pred_dists =  np.array(outputs[2][0,1,:,:].asnumpy()) 
                pred_segm = 1-pred_segm
                #print("Shape of pred_segm:", pred_segm.shape, "Shape of pred_bound:", pred_bound.shape, "Shape of binary_extent_mask:", pred_segm.shape)
                #print("NaNs in pred_segm:", np.isnan(pred_segm).sum(), "NaNs in binary_extent_mask:", np.isnan(pred_segm).sum())
                inst =InstSegm(pred_segm, pred_bound, t_ext=t_ext, t_bound=t_bound)   # perform instance segmentation
                #print("NaNs in inst:", np.isnan(inst).sum())
                inst = np.nan_to_num(inst, nan=0)
                if is2022:
                    imgColl = plotPredictedImape(id, img, mask, pred_segm, pred_bound, pred_dists, inst, ref_path)
                    plotColl.append(imgColl)
                output_shapefile_path = visualize_all(id, img, currentMetadata, outputs, pred_segm, pred_bound, inst, result_path)
                print("Start IOU calculation")
                csv_file_path = os.path.join(result_path, str(id), "iou.csv")
                iou_score= get_iou(os.path.join(ref_path, f'tile_{id}.shp'), os.path.join(output_shapefile_path, f'{str(id)}.shp'))
                ious.append({ "ID": id,"IOU": iou_score })
                saveAsCSV(["ID", "IOU"], csv_file_path, ious, True)
        except Exception as e:
            print(f"Error processing image ID {id}: {e}")
    
    return plotColl
       
def visualize(result_path, val_ids,  t_ext , t_bound , is2022 = True):
    random_val_ids = random.choice(val_ids) # Choose a random validation ID
    return visualize_predictions(result_path, val_ids ,t_ext = t_ext, t_bound = t_bound, is2022 = is2022)
    


# Applying model in 2022 images

In [ ]:
#This is for vnir: t_ext = 0.6, t_bound = 0.1
#6612
results_2022 = visualize(result_path, val_ids, t_ext = 0.6, t_bound = 0.1, is2022 = True)

In [ ]:
plotIOUS(r"D:\Source\Output\Result_2022\VNIR\648\result\6614\iou.csv", r"D:\Source\Output\Result_2022\VNIR\648\iou_Hist_2022.png")

In [ ]:
plotIOUS(r"D:\Source\Output\Result_2022\NDV\648\result\6782\iou.csv", r"D:\Source\Output\Result_2022\NDV\648\result_ndv_2022.png")

In [ ]:
plotIOUS(r"D:\Source\Output\Result_2010\VNIR\99\iou.csv", r"D:\Source\Output\Result_2010\VNIR\IOU_2010.png")

In [ ]:
plot_images_with_masks(results_2022[:3] , r"D:\Source\Output\Result_2022\NDV\648\results_ovelap_2022.png")

In [ ]:
plot_images_with_masks(results_2022[:3] , r"D:\Source\Output\Result_2022\VNIR\648\results_2022.png")

# Applying model in 2010 images

In [ ]:
#list(image_2010_dict.keys())
results_2010 =visualize(output_directory_2010, list(image_dict_2010.keys()),  t_ext = 0.6, t_bound = 0.1, is2022= False)